<a href="https://colab.research.google.com/github/Muhammadridho100902/google_collab/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! chmod 600 /content/kaggle.json
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 168MB/s]


In [1]:
import zipfile
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import pickle

In [3]:
dataset_zip = zipfile.ZipFile('twitter-entity-sentiment-analysis.zip')
dataset_zip.extractall()
dataset_zip.close()

In [4]:
def getOneHotEncode(label):
  # penggunaan lower untuk meng antisipasi label yang berbeda seperti "Positive" & "positive"
  if label.lower() == 'positive':
    return [1,0,0]
  elif label.lower() == 'negative':
    return [0,1,0]
  else:
    return [0,0,1]

In [5]:
nltk.download('stopwords')
stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
def remove_url (text):
  # http://google.com adalah bla bla bla
  # https://youtube.com
  return re.sub(r'http\S+', '', text)

def remove_number(text):
  # 123 saya blablabla
  return re.sub(r'\d+', '', text)

def remove_stopwords(text):
  words = text.split() # saya mau ke batam -> [saya, mau, ke, batam]
  new_text = ""
  for word in words:
    if word not in stopwords.words('english'):
      new_text += word
      new_text += " "

  return new_text

def lematized_text(text):
  lematizer = WordNetLemmatizer()
  words = text.split()
  new_text = ""
  for word in words:
    lematized_word = lematizer.lemmatize(word)
    new_text += lematized_word
    new_text += " "
  return new_text

def stemmed_text(text):
  stemmer = PorterStemmer()
  words = text.split()
  new_text = ""
  for word in words:
    stemmed_word = stemmer.stem(word)
    new_text += stemmed_word
    new_text += " "
  return new_text

def preprocessing (text):
  #lowercase -> Running, running, RUNNING, RunNinG
  text = text.lower()

  # remove url
  text = remove_url(text)

  # remove number
  text = remove_number(text)

  # remove stopwords
  text = remove_stopwords(text)

  # lematisasi
  text = lematized_text(text)

  # stemming
  text = stemmed_text(text)

  return text

In [7]:
text = []
label = []

with open('/content/twitter_training.csv', 'r') as file:
  for sentence in file:
    values = sentence.split(',')
    text.append(values[3])
    label.append(getOneHotEncode(values[2]))

In [8]:
print(text[7])
print(label[7])

"So I spent a couple of hours doing something for fun... If you don't know that I'm a huge @ Borderlands fan and Maya is one of my favorite characters
[1, 0, 0]


# Tokenizing

proses untuk mengubah teks / kata menjadi sebuah token agar bisa diterima oleh machine

In [9]:
def get_avg(text_list):
  sum = 0
  for text in text_list:
    word_num = len(text)
    sum = sum + word_num
  return sum / len(text_list) # total kata di dataset / banyak kalimat yang ada

In [10]:
def tokenizing(text_list):
  vocab_size = 10000

  # penggunaan oov bertujuan untuk handle kata yang lebih dari word tokenizer
  # contoh kita set vocab size di 2
  # kita ingin membuat kata "aku adalah budi" menjadi token
  # maka "Aku" menjadi index 1 "adalah" menjadi index 2 dan "budi" akan menjadi OOV
  # artinya adalah oov merupakan sebuah param untuk handle sebuah kata yang lebih dari vocab size yang kita set

  tokenizer = Tokenizer(num_words= vocab_size, oov_token = '<OOV>')
  tokenizer.fit_on_texts(text_list)
  word_index = tokenizer.word_index # mengubah kata menjadi index

  max_length = get_avg(text_list)
  max_length = int(max_length)
  print(max_length)

  squence = tokenizer.texts_to_sequences(text_list)

  # pad_squence ini berguna untuk menambahkan atau memotong ketika kita menggunakan shape pada tensorflow
  # ada dua yaitu padding dan trunct
  ## padding untunk menambahkan
  # ketika shapenya 4 dan value kita 3 ([1,2,3]) jika mengunakan padding matrix akan bertambah ([1,2,3,4])
  ## trunct untuk memotong
  # ketika shapenya 4 dan value kita 5 ([1,2,3,4,5]) jika mengunakan trunct matrix akan berkurang ([1,2,3,4])


  # setiap kalimat memiliki jumlah kata yang berbeda. untuk handle hal tersebut, kita menggunakan maxlen
  # agar setiap kalimat memiliki jumlah yang sama
  # saat ini kita menggunakan 100 kata dalam setiap kalimat
  # jika di suatu kalimat memiliki 3 kata saja
  # maka squence akan bertambah sampai berjumlah 100
  # akan diisi dengan nilai 0
  # pada saat ini kita menggunakan post, karena akan menambahkan nilai 0 setelah squence sampai berjumlah 100 kata

  padded_squence = pad_sequences(squence, padding = 'pre', truncating= 'pre', maxlen = max_length)

  # export tokenizer
  with open('my_tokenizer.pickle', 'wb') as file:
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

  return padded_squence

In [11]:
datas = tokenizing(text)
datas.shape

83


(74682, 83)

In [12]:
labels = np.array(label) # labelnya diubah menjadi array terlebih dahulu

In [13]:
from tensorflow.keras.layers import Embedding

model = tf.keras.Sequential([
    # input shape mengikuti datas.shape, karna kita ingin membuat datanya fleksibel makanya kita set None
    # jika dilihat data diatas berjumlah 74682
    tf.keras.layers.Embedding(10001, output_dim=100, input_length = 83),

    tf.keras.layers.LSTM(100),

    tf.keras.layers.Dense(100, activation = 'relu', input_shape=(None, 75)),

    # output set di 3 karna akan menghasilkan antara "Pos", "Neg", "Neu"
    tf.keras.layers.Dense(3, activation = 'softmax')
])

In [14]:
model.compile(
    loss = 'categorical_crossentropy',
    metrics = ['acc'],
    optimizer = 'adam'
)

In [15]:
model.fit(
    datas,
    labels,
    epochs = 3
)

Epoch 1/3
2334/2334 [==============================] - 196s 83ms/step - loss: 0.6867 - acc: 0.6986
Epoch 2/3
2334/2334 [==============================] - 193s 82ms/step - loss: 0.4062 - acc: 0.8343
Epoch 3/3
2334/2334 [==============================] - 194s 83ms/step - loss: 0.2959 - acc: 0.8796


In [16]:
model.save('my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import tensorflow as tf
import pickle

from tensorflow.keras.models import load_model

In [3]:
with open('/content/my_tokenizer.pickle', 'rb') as file:
  tokenizer = pickle.load(file)

In [ ]:
tokenizer.word_index

In [4]:
model = load_model('my_model.h5')

with open('/content/my_tokenizer.pickle', 'rb') as file:
  tokenizer = pickle.load(file)

In [5]:
nltk.download('stopwords')
stopwords.words('english')
nltk.download('wordnet')

def remove_url (text):
  # http://google.com adalah bla bla bla
  # https://youtube.com
  return re.sub(r'http\S+', '', text)

def remove_number(text):
  # 123 saya blablabla
  return re.sub(r'\d+', '', text)

def remove_stopwords(text):
  words = text.split() # saya mau ke batam -> [saya, mau, ke, batam]
  new_text = ""
  for word in words:
    if word not in stopwords.words('english'):
      new_text += word
      new_text += " "

  return new_text

def lematized_text(text):
  lematizer = WordNetLemmatizer()
  words = text.split()
  new_text = ""
  for word in words:
    lematized_word = lematizer.lemmatize(word)
    new_text += lematized_word
    new_text += " "
  return new_text

def stemmed_text(text):
  stemmer = PorterStemmer()
  words = text.split()
  new_text = ""
  for word in words:
    stemmed_word = stemmer.stem(word)
    new_text += stemmed_word
    new_text += " "
  return new_text

def preprocessing (text):
  #lowercase -> Running, running, RUNNING, RunNinG
  text = text.lower()

  # remove url
  text = remove_url(text)

  # remove number
  text = remove_number(text)

  # remove stopwords
  text = remove_stopwords(text)

  # lematisasi
  text = lematized_text(text)

  # stemming
  text = stemmed_text(text)

  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
text = input('Masukan text yang mau di predict : ')

Masukan text yang mau di predict : you give me so much loved, but you make me sad


In [7]:
preprocessed_text = preprocessing(text)

In [8]:
print(text)
print(preprocessed_text)

you give me so much loved, but you make me sad
give much loved, make sad 


In [9]:
sequence = tokenizer.texts_to_sequences([preprocessed_text])
print(sequence)

[[273, 124, 663, 139, 444]]


In [10]:
padded_sequences = pad_sequences(sequence, padding = 'pre', truncating= 'pre', maxlen = 83)

In [11]:
print(padded_sequences)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 273 124 663 139 444]]


In [14]:
result = model.predict(padded_sequences)

1/1 [==============================] - 0s 106ms/step


In [16]:
output_array = np.array(result)

In [19]:
print(output_array)

[[0.09125289 0.8945227  0.01422437]]


In [17]:
predicted_category_index = np.argmax(output_array)

In [20]:
# Mendefinisikan kategori
categories = ["Positive", "Negative", "Neutral"]

In [21]:
# Mendapatkan kategori berdasarkan indeks
predicted_category = categories[predicted_category_index]

In [22]:
print(f"Predicted category: {predicted_category}")

Predicted category: Negative


# Use to WebAPP

In [59]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

def predictedSentences():
    text = input("Masukan Kalimat untuk di Prediksi: ")
    def remove_url(text):
        # http://google.com adalah bla bla bla
        # https://youtube.com
        return re.sub(r'http\S+', '', text)

    def remove_number(text):
        # 123 saya blablabla
        return re.sub(r'\d+', '', text)

    def remove_stopwords(text):
        words = text.split()
        new_text = ""
        for word in words:
            if word not in stopwords.words('english'):
                new_text += word
                new_text += " "

        return new_text

    def lemmatized_text(text):
        lemmatizer = WordNetLemmatizer()
        words = text.split()
        new_text = ""
        for word in words:
            lemmatized_word = lemmatizer.lemmatize(word)
            new_text += lemmatized_word
            new_text += " "
        return new_text

    def stemmed_text(text):
        stemmer = PorterStemmer()
        words = text.split()
        new_text = ""
        for word in words:
            stemmed_word = stemmer.stem(word)
            new_text += stemmed_word
            new_text += " "
        return new_text

    def preprocessing(text):
        # lowercase -> Running, running, RUNNING, RunNinG
        text = text.lower()

        # remove url
        text = remove_url(text)

        # remove number
        text = remove_number(text)

        # remove stopwords
        text = remove_stopwords(text)

        # lemmatization
        text = lemmatized_text(text)

        # stemming
        text = stemmed_text(text)

        return text

    preprocessed_text = preprocessing(text)
    return preprocessed_text

In [66]:
def predictSentiment():
  model = load_model('my_model.h5')

  with open('/content/my_tokenizer.pickle', 'rb') as file:
    tokenizer = pickle.load(file)

  preprocessed_text = predictedSentences()
  sequence = tokenizer.texts_to_sequences([preprocessed_text])
  padded_sequences = pad_sequences(sequence, padding = 'pre', truncating= 'pre', maxlen = 83)
  result = model.predict(padded_sequences)
  output_array = np.array(result)
  predicted_category_index = np.argmax(output_array)
  # Mendefinisikan kategori
  categories = ["Positive", "Negative", "Neutral"]
  # Mendapatkan kategori berdasarkan indeks
  predicted_category = categories[predicted_category_index]
  print(f"Predicted category: {predicted_category}")

In [67]:
predictSentiment()

Masukan Kalimat untuk di Prediksi: you give me so much loved, but you make me sad
1/1 [==============================] - 0s 456ms/step
Predicted category: Negative
